In [2]:

import math, re, ccxt
from pprint import pformat
wex = ccxt.wex()
data_for_graph = {}
data = wex.fetch_tickers()
for key in data:
    data_for_graph.update({key: data[key]['last']})
ticker_list = list(data_for_graph.keys())
ticker_list2 = []
for key in ticker_list:
    ticker_list2.append("{1}/{0}".format(*key.split('/'))) #обратные тикеры

for key in range(0, len(ticker_list)):
    #обратные котировки
    data_for_graph.update({ticker_list2[key]: 1 / (data[ticker_list[key]]['last'])})

print(pformat(data_for_graph))


{'BCH/BCHET': 1.519756838905775,
 'BCH/BTC': 0.1001,
 'BCH/DASH': 2.186,
 'BCH/ETH': 1.745,
 'BCH/EUR': 580.544,
 'BCH/LTC': 5.811,
 'BCH/RUR': 40404.064,
 'BCH/USD': 710.591,
 'BCH/ZEC': 3.681,
 'BCHET/BCH': 0.658,
 'BTC/BCH': 9.990009990009991,
 'BTC/BTCET': 1.6,
 'BTC/DASH': 21.867483052700635,
 'BTC/ETH': 17.36714136853074,
 'BTC/EUR': 5776.85614,
 'BTC/LTC': 58.07200929152149,
 'BTC/NMC': 4761.9047619047615,
 'BTC/NVC': 2127.6595744680853,
 'BTC/PPC': 4347.826086956522,
 'BTC/RUR': 405830.6975,
 'BTC/USD': 7123.39,
 'BTC/ZEC': 36.76470588235294,
 'BTCET/BTC': 0.625,
 'DASH/BCH': 0.4574565416285453,
 'DASH/BTC': 0.04573,
 'DASH/DSHET': 1.3793103448275863,
 'DASH/ETH': 0.798,
 'DASH/EUR': 264.885,
 'DASH/LTC': 2.651,
 'DASH/RUR': 18530.695,
 'DASH/USD': 326.48011,
 'DASH/ZEC': 1.683,
 'DSHET/DASH': 0.725,
 'ETH/BCH': 0.5730659025787965,
 'ETH/BTC': 0.05758,
 'ETH/DASH': 1.2531328320802004,
 'ETH/ETHET': 1.6722408026755853,
 'ETH/EUR': 329.73244,
 'ETH/LTC': 3.32513,
 'ETH/RUR': 2320

In [ ]:
'''

Создаем граф в виде списка смежности для каждой ноды - тикера криптовалюты, 
котировки переводим в логарифмический вид, чтобы можо было складывать вместо 
умножения
'''


In [3]:

graph = {}
jsrates = data_for_graph

pattern = re.compile("([A-Z]{3,5})/([A-Z]{3,5})")

try:
    for key in jsrates:
        matches = pattern.match(key)

        conversion_rate = math.log(float(jsrates[key]))
        
        from_rate = matches.group(1).encode('ascii', 'ignore')

        to_rate = matches.group(2).encode('ascii', 'ignore')

        if from_rate != to_rate:
            if from_rate not in graph:
                graph[from_rate] = {}
            graph[from_rate][to_rate] = float(conversion_rate)

except AttributeError:
    print('Strange rate')
    pass

print(pformat(graph))
# воплощение алгоритма отсюда
#  https://gist.github.com/joninvski/701720


{b'BCH': {b'BCHET': 0.41855034765681975,
          b'BTC': -2.3015855926609623,
          b'DASH': 0.7820733897543468,
          b'ETH': 0.5567545556543906,
          b'EUR': 6.363965594992479,
          b'LTC': 1.7597526730929685,
          b'RUR': 10.606685652949404,
          b'USD': 6.5660970181534974,
          b'ZEC': 1.303184454396884},
 b'BCHET': {b'BCH': -0.4185503476568198},
 b'BTC': {b'BCH': 2.3015855926609623,
          b'BTCET': 0.47000362924573563,
          b'DASH': 3.0850007413121734,
          b'ETH': 2.854579993798107,
          b'EUR': 8.661614893272828,
          b'LTC': 4.061683779982553,
          b'NMC': 8.468403027246806,
          b'NVC': 7.66277786326017,
          b'PPC': 8.377431249041079,
          b'RUR': 12.91369135038315,
          b'USD': 8.871139014690653,
          b'ZEC': 3.6045383056801854},
 b'BTCET': {b'BTC': -0.4700036292457356},
 b'DASH': {b'BCH': -0.7820733897543467,
           b'BTC': -3.0850007413121734,
           b'DSHET': 0.321583624127462

In [ ]:

'''
Подготавливаем данные по каждой вершине графа 
и ищем отрицательный цикл в полученном графе

'''

In [4]:


# Step 1: For each node prepare the destination and predecessor
def initialize(graph, source):
    d = {}  # Stands for destination
    p = {}  # Stands for predecessor
    for node in graph:
        d[node] = float('Inf')  # We start admiting that the rest of nodes are very very far
        p[node] = None
    d[source] = 0  # For the source we know how to reach
    return d, p


def relax(node, neighbour, graph, d, p):
    # If the distance between the node and the neighbour is lower than the one I have now
    if d[neighbour] > d[node] + graph[node][neighbour]:
        # Record this lower distance
        d[neighbour] = d[node] + graph[node][neighbour]
        p[neighbour] = node


def retrace_negative_loop(p, start):
    arbitrageLoop = [start]
    next_node = start
    while True:
        next_node = p[next_node]
        if next_node not in arbitrageLoop:
            arbitrageLoop.append(next_node)
        else:
            arbitrageLoop.append(next_node)
            arbitrageLoop = arbitrageLoop[arbitrageLoop.index(next_node):]
            return arbitrageLoop


def bellman_ford(graph, source):
    d, p = initialize(graph, source)
    for i in range(len(graph) - 1):  # Run this until is converges
        for u in graph:
            for v in graph[u]:  # For each neighbour of u
                relax(u, v, graph, d, p)  # Lets relax it

    # Step 3: check for negative-weight cycles
    for u in graph:
        for v in graph[u]:
            if d[v] < d[u] + graph[u][v]:
                return (retrace_negative_loop(p, source))
    return None


In [ ]:

'''
Применяем алгоритм поиска и выводим ноды, 
входящие в отрицательный цикл, если он существует.
Расчитываем его стоимость с учетом 0.2% комиссии
'''

In [7]:
 paths = []
 for key in graph:
    
    path = bellman_ford(graph, key)

    if path not in paths and not None and len(path) > 3:
        paths.append(path)

for path in paths:
    if path == None:
        print("No opportunity here :(")
    else:
        print(len(path))
        money = 100
        print("Starting with %(money)i in %(currency)s" % {"money": money, "currency": path[0]})           

        for i, value in enumerate(path):
            if i + 1 < len(path):
                start = path[i]
                end = path[i + 1]
                rate = math.exp(graph[start][end])
                money *= rate*0.998
                print("%(start)s to %(end)s at %(rate)f = %(money)f" % {"start": start, "end": end, "rate": rate,
                                                                        "money": money})
print ("\n")

 

5
Starting with 100 in b'BTC'
b'BTC' to b'DASH' at 21.867483 = 2182.374809
b'DASH' to b'USD' at 326.480110 = 711076.963657
b'USD' to b'NVC' at 0.301659 = 214073.849089
b'NVC' to b'BTC' at 0.000470 = 100.413480
5
Starting with 100 in b'DASH'
b'DASH' to b'USD' at 326.480110 = 32582.714978
b'USD' to b'NVC' at 0.301659 = 9809.215550
b'NVC' to b'BTC' at 0.000470 = 4.601111
b'BTC' to b'DASH' at 21.867483 = 100.413480
5
Starting with 100 in b'USD'
b'USD' to b'NVC' at 0.301659 = 30.105581
b'NVC' to b'BTC' at 0.000470 = 0.014121
b'BTC' to b'DASH' at 21.867483 = 0.308180
b'DASH' to b'USD' at 326.480110 = 100.413480
5
Starting with 100 in b'NVC'
b'NVC' to b'BTC' at 0.000470 = 0.046906
b'BTC' to b'DASH' at 21.867483 = 1.023665
b'DASH' to b'USD' at 326.480110 = 333.537761
b'USD' to b'NVC' at 0.301659 = 100.413480




In [23]:
'''
Для bid/ask

'''

wex = ccxt.wex()
data_for_graph = {}
data = wex.fetch_tickers()
for key in data:
    data_for_graph.update({key: data[key]['ask']})
ticker_list = list(data_for_graph.keys())
ticker_list2 = []
for key in ticker_list:
    ticker_list2.append("{1}/{0}".format(*key.split('/'))) #обратные тикеры

for key in range(0, len(ticker_list)):
    #обратные котировки
    data_for_graph.update({ticker_list2[key]: 1 / (data[ticker_list[key]]['bid'])})

#print(pformat(data_for_graph))

graph OK


In [24]:

graph = {}
jsrates = data_for_graph

pattern = re.compile("([A-Z]{3,5})/([A-Z]{3,5})")

try:
    for key in jsrates:
        matches = pattern.match(key)

        conversion_rate = math.log(float(jsrates[key]))
        
        from_rate = matches.group(1).encode('ascii', 'ignore')

        to_rate = matches.group(2).encode('ascii', 'ignore')

        if from_rate != to_rate:
            if from_rate not in graph:
                graph[from_rate] = {}
            graph[from_rate][to_rate] = float(conversion_rate)

except AttributeError:
    print('Strange rate')
    pass

print('graph OK')


graph OK


In [22]:
paths = []
for key in graph:
    path = bellman_ford(graph, key)

    if path not in paths and not None and len(path) > 3:
        paths.append(path)

for path in paths:
    if path == None:
        print("No opportunity here :(")
    else:
        print(len(path))
        money = 100
        print("Starting with %(money)i in %(currency)s" % {"money": money, "currency": path[0]})

        for i, value in enumerate(path):
            if i + 1 < len(path):
                start = path[i]
                end = path[i + 1]
                rate = math.exp(graph[start][end])
                money *= rate * 0.998
                print("%(start)s to %(end)s at %(rate)f = %(money)f" % {"start": start, "end": end, "rate": rate,
                                                                        "money": money})
print("\n")


graph OK
